각 time-step의 디코더 출력값과 어텐션 결과값을 이어붙인 후에 생성자 모듈에서 `softmax`를 취하여 확률 분포를 구합니다.

<br></br>
![](./images/10-4-1-inputfeeding.jpg)
<br></br>

이후에 해당 확률 분포에서 argmax를 수행하여 $\hat{y_t}$를 샘플링합니다. 하지만 분포에서 원핫벧터로 변환하는 과정에서 많은 정보가 손실됩니다. 따라서 단순히 다음 time-step에 $\hat{y_t}$을 입력으로 넣어주는 것보다는 `softmax` 이전의 값도 같이 넣어주어 게산하는 편이 정보의 손실 없이 더 좋은 효과를 얻는 방법입니다.

y와 달리 concat 계층의 출력은 y가 임베딩 계층에서 dense 벡터로 변환되고 난 후에 임베딩 벡터와 이어붙여 디코더 RNM에 입력으로 주어집니다.

이런 과정을 **input feeding**이라고 합니다.

<br></br>
$$
h_t^{src} = RNN_{enc}(emb_{src}(x_t),h_{t-1}^{src}) \\
H^{src} = [h_1^{src}; h_2^{src}; \dots; h_n^{src}] \\
h_t^tgt = RNN_{dec}([emb_{tgt}(y_{t-1};\tilde{h_{t-1}^{tgt}}], h_{t-1}^{tgt}) \\
\text{where}\ h_0^{tgt} = h_n^{src}, y_0 = BOS \\
w = softmax(h_t^{tgt}WH^{srcT}) \\
c = wH^{src}\ \text{and c is a context vector} \\
h_t^{tgt} = tanh(linear_{2hs \to hs}([h_t^{tgt};c])) \\
\hat{y_T} = softmax(linear_{hs \to |V_{tgt}|}(\tilde{h_t^{tgt}}) \\
\text{where hs is hidden size of RNN and |V| is size of output vocabulary.}
$$
<br></br>

이 수식은 어텐션과 input feeding이 추가된 seq2seq의 처음부터 끝까지를 수식으로 나타낸 것입니다. $RNN_{dec}$는 이제 $\tilde{h_{t-1}^{tgt}}$를 입력으로 받으므로, 모든 time-step을 한번에 처리하도록 구현할 수 없다는 점이 구현상의 차이점입니다.

### 단점

이 방식은 훈련속도 저하라는 단점을 가집니다. input feeding 이전의 방식에서는 훈련할대, 인코더와 마찬가지로 디코더도 모든 time-step에 대해 한번에 계산할 수 있었습니다. 하지만 input feeding으로 인해 디코더 RNN의 입력으로 이전 time-step의 결과 $\tilde{h_t^{tgt}}$가 필요하게 되어 각 time-step별로 순차적으로 계산해야 합니다.

하지만 이 단점이 크게 부각되지 않는 이유는 어차피 추런 단계에서는 디코더는 input feeding이 아니더라도 time-step별로 순차적으로 계산되어야 하기 때문입니다. 추론 단계에서는 이전 time-step의 출력값인 $\hat{y_t}$를 디코더 (정확하게는 디코더 RNN이전의 임베딩 계층)의 입력으로 사용해야 하므로, 어쩔수 없이 병렬 처리가 아닌 순차적으로 계산해야 합니다

### 성능 개선

어텐션과 input feeding을 사용함으로써 최종적으로 훨씬 더 나은 성능을 얻을 수 있음을 알 수 있습니다.

<br></br>

|번호|모델|알고리즘|PPL|BLEU|
|---|---|------|---|----|
|1|Base|seq2seq|10.6|11.3|
|2|1+reverse|BiDirectional LSTM Encoder|9.9|12.6|
|3|2+dropout|Dropout (0.2)|8.1|14.0|
|4|3+attention|Attention|7.3|16.8|
|5|4+input feed|input feeding 추가|6.4|18.1|

<br></br>

### 구현 관점에서 바라보기

구현 관점에서 seq2seq 수식을 따라가보겠습니다.

다음과 같이 소스 문장과 타깃 문장으로 이루어진 병렬 코퍼스가 있습니다.

<br></br>
$$
B = \{X_i, Y_i\}_{i=1}^N \\
\text{where}\ X = \{x_1, \dots, x_n \}, Y = \{y_1, \dots, y_m \}
$$
<br></br>

소스 문장은 n개 단어로 이루어져있고, 타겟 문장은 m개의 단어로 이루어져 있습니다. 

이때 미니배치단위로 seq2seq의 훈련이 어떻게 이루어지는지 살펴보겠습니다. 각 소스 문장과 타깃 문장의 미니배치 크기는 다음과 같습니다.

<br></br>
$$
|X| = (batch size, n |V_{src}|) \\
|x_t| = (batch size, 1, |V_{src}|) \\
|Y| = (batch size, m, |V_{tgt}|) \\
|y_t| = (batch size, 1, |V_{tgt}|)
$$
<br></br>

인코더에 1 time-step을 넣고 피드포워드하는 과정은 다음과 같습니다.

하나의 단어 또는 토큰을 임베딩 계층에 통과시켜 정해진 단어 임베딩 벡터 크기의 텐서를 얻습니다. 이것을 다시 이전 time-step의 RNN의 은닉 상태와 함께 RNN에 입력으로 넣으면, 현재 time-step RNN의 은닉 상태가 나옵니다. RNN이 1의 계층만 가지고 있다면, 은닉 상태가 곧 출력값이 됩니다. 만약 RNN이 여러개의 계층을 갖고 있다면, 마지막 계층의 은닉 상태가 출력값이 됩니다.

지금은 RNN이 1개의 계층만 가지고 있다고 가정하고, 은닉 상태가 곧 출력값이 될 것입니다.

<br></br>
$$
h_t^{src} = RNN_{enc}(emb_{src}(x_t),h_{t-1}^{src}) \\
|emb_{src}(x_t)| = (batch\ size, 1, word\ vec\ dim) \\
|h_t^{src}| = (batch\ size, 1, hidden\ size)
$$
<br></br>

이것을 전체 time-step에 대해 인코더에서 병렬로 수행할 수 있습니다. 이는 파이토치로 쉽게 구현할 수 있습니다.

<br></br>
$$
H^{src} = [h_1^{src}; h_2^{src}; \dots ; h_n^{src}] \\
= RNN_{enc}(emb_{src}(X),h_0^{src})
$$
<br></br>

전체 time-step의 RNN의 출력 텐서의 크기는 다음과 같습니다. 이는 각 time-step의 RNN의 은닉 상태들을 순서 차원에 대해 concat한 것과 같습니다. 따라서 1 time-step일때의 텐서 크기보다 n배 커졌습니다.

<br></br>
$$
|H^{src}| = (batch\ size, n, hidden\ size)
$$
<br></br>

디코더의 피드포워드에 관해 다루겠습니다. 마찬가지로 1 time-step의 단어 또는 코튼을 타깃 언어의 임베딩 계층에 통과시키면, 정해딘 단어 임베딩 벡터 크기의 텐서를 얻습니다. 이것을 다시 디코더의 RNN에 이전 time-step의 은닉 상태와 함께 넣어주어 현재 time-step의 은닉 상태를 구합니다. 주의할 점은 RNN에 넣어줄대, 이전 time-step의 $\tilde{h_{t-1}^{tgt}}$를 단어 임베딩 텐서에 이어붙여 넣어준다는 것입니다. 따라서 RNN의 입력의 크기에 주목해야합니다. 또한, 첫번째 time-step의 디코더 입력 단어는 BOS로 문장의 시작을 가르키며, 첫번째 time-step의 디코더 RNN 은닉 상태는 인코더 RNN의 마지막 time-step의 은닉 상태입니다.

<br></br>
$$
h_t^{src} = RNN_{dec}([emb_{tgt}(y_{t-1});\tilde{h_{t-1}^{tgt}}],h_{t-1}^{tgt}) \\
\text{where}\ h_0^{tgt} = h_n^{src}, y_0 = BOS \\
|emb_{src}(y_{t-1})| = (batch\ size, 1, word\ vec\ dim) \\
|h_t^{tgt}| = (batch\ size, 1, hidden\ size) \\
|[emb_{tgt}(y_{t-1});\tilde{h_{t-1}^{tgt}}]| = (batch\ size, 1, word\ vec\ dim\ +\ hidden\ size)
$$
<br></br>

어텐션의 구현 부분의 수식을 따라가보겠습니다. 다음 수식에서 곱하기 부호 x는 실제 곱하기가 아닌, 좌우의 크기를 가진 텐서 사이의 행렬 곱입니다. 기본적으로 행렬곱은 2차원의 행렬에 대해서만 정의되며, 이는 보통 마지막 2개 차원에 대해이루어 집니다.

다음은 Attention 가중치를 구하는 수식입니다.

<br></br>
$$
w = softmax(H^{src}(h_t^{tgt}W)) \\
|W| = (hidden\ size, hidden\ size) \\
\\
|h_t^{tgt}W| = (batch\ size, 1, hidden\ size) X (hidden\ size, hidden\ size) \\
= (batch\ size, 1, hidden\ size) \\
\\
|H^{src}(h_t^{tgt}W)| = (batch\ size, n, hidden\ size) X (batch\ size, hidden\ size, 1) \\
= (batch\ size, n, 1) \\
= (batch\ size, 1, n)
$$
<br></br>

어텐션 가중치 텐서에 주목합시다. 첫번째 차원은 미니배치 내의 샘플의 순서 인덱스를 가르킵니다. 두번째 차원은 문장 내 time-step을 의미하는데, 1이 들어있어 타깃 문장의 1개 time-step임을 알 수 있습니다. 마지막 세번째 차원은 n이 들어있어, 소스 문장의 전체 time-step임을 알 수 있습니다. 즉, 어텐션 가중치 텐서가 의미하는 값은

* 미니배치의 각 샘플별로
* 디코더의 현재 time-step에 관해
* 인코더의 각각 time-step별 어텐션 가중치 값

을 가지게 됩니다.

다음 수식은 앞에서 얻은 어텐션 가중치에 인코더 RNN 전체 time-step의 출력 텐서를 곱하여, 가중치에 따라 가중합을 구하는 과정입니다. 이렇게 구해진 텐서는 Context 텐서 c가 됩니다.

<br></br>
$$
c = wH^{src} \\
|c| = (batch\ size, 1, n) X (batch\ size, n, hidden\ size) \\
= (batch\ size, 1, hidden\ size)
$$
<br></br>

컨텍스트 텐서와 현재 time-step의 디코더의 출력값을 이어붙여 원래 **hidden_size**의 2배가 되는 텐서를 얻습니다.

이를 다음과 같은 Linear layer에 넣고 tanh를 통과시켜 원래의 hidden_size를 갖는 텐서로 만들어줍니다. 이를 우리는 $\tilde{h_t^{tgt}}$라고 부르며, 다음 time-step의 디코더의 또 다른 입력값으로 주어 input feeding을 구현합니다.

<br></br>
$$
\tilde{h_t^{tgt}}  = tanh(linear_{2hs \to hs}([h_t^{tgt};c])) \\
|[h_t^{tgt}; c]| = (batch\ size, 1, hidden\ size + hidden\ size) \\
= (batch\ size, 1, 2\ x\ hidden\ size)
$$
<br></br>

<br></br>
$$
linear_{2hs \to hs}(x) = Wx + b \\
\text{where}\ W \in R^{\text{2 hidden_size x hidden_size}}, b \in R^{hidden\ size} \\
|\tilde{h_t^{tgt}}| = (batch\ size, 1, hidden\ size)
$$
<br></br>

앞에서 얻은 $\tilde{h_t^{tgt}}$를 소프트맥스 계층을 통과시켜, 소스 문장과 이전 time-step의 타깃 단어들이 주어졌을때 현재 time-step의 탁딧 단어에 대한 확률 분포를 구합니다. 이는 불연속적인 멀티눌리 확률 분포입니다. 즉, 타깃 언어의 각 단어별 확률값을 가진 계층이 됩니다. 여기서 최고 확률값을 갖는 단어를 $\hat{y_t}$라고 합니다.

<br></br>
$$
P(y_t|X, y_{<t};\theta) = softmax(linear_{hs \to |V_{tgt}}|(\tilde{h_t^{tgt}})) \\
\hat{y_t} = argmax_{y \in Y}P(y_t|X,y_{<t};\theta) \\
linear_{hs \to |V_{tgt}|}(x) = Wx + b \\
\text{where}\ W \in R^{hidden\ size\ x\ |V_{tgt}|}, b \in R^{|V_{tgt}|} 
$$
<br></br>

따라서 현재 time-step의 타깃 단어 텐서의 크기는 다음과 같습니다.

<br></br>
$$
|\hat{y_t}| = (batch\ size, 1, |V_{tgt}|)
$$
<br></br>

다음은 seq2seq 신경망 파라미터 $\theta$를 훈련하는 손실 함수를 살펴보겠습니다. 교차 엔트로피 소 ㄴ실 함수를 사용하면 다음과 같은 수식이 만들어질 것입니다. 이전에 설명했듯이 $y_t^i$는 원핫 벡터이므로, 정답 단어 인덱스만 1이고 나머지는 0으로 채워져 있습니다. 따라서 예측한 확률 분포에서 정답 단어 인덱스의 확률값에 대한 평균 곱하기 -1인것과 같습니다.

<br></br>
$$
L(\theta) = - \frac{1}{N} \sum_{i=1}^N y_t^T log P(y_t|X,y_{<t};\theta) \\
|y_t log P(y_t|X_i,y_{<t};\theta)^T| = (batch\ size, 1, |V_{tgt}|) x (batch\ size, 1, |V_{tgt}|)^T \\
= (batch\ size, 1, |V_{tgt}|) x (batch\ size, |V_{tgt}|, 1) \\
= (batch\ size, 1, 1) \\
= (batch\ size,)
$$
<br></br>

미니배치 내의 샘플별 손실값이 구해지면, 이를 모두 평균내어 scalar로 만들고 -1을 곱합니다. 이후 손실값을 $\theta$에 관해 미분하여 경사하강법을 수행하면 seq2seq의 파라미터를 업데이트할 수 있습니다.

<br></br>
$$
\theta \gets \theta - \gamma \nabla_\theta L_\theta(\hat{Y},Y)
$$
<br></br>

### 파이토치 예제

#### Attention Class

In [2]:
import torch
import torch.nn as nn

class Attention(nn.Module):
    
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        
        self.linear = nn.Linear(hidden_size, hidden_size, bias = False)
        self.softmax = nn.Softmax(dim = -1)
    
    def forward(self, h_src, h_t_tgt, mask = None):
        ## |h_src| = (batch_size, length, hidden_size)
        ## |h_t_tgt| = (batch_size, 1, hidden_size)
        ## |mask| = (batch_size, length)
        
        ## |query| = (batch_size, hidden_size, 1)
        query = self.linear(h_t_tgt.squeeze(1)).unsqueeze(-1)
        
        ## |weight| = (batch_size, length)
        weight = torch.bmm(h_src, query).squueze(-1)
        
        if mask is not None:
            ## Set each weight as -inf, if the mask value equals to 1
            ## Since the softmax operation makes -inf to 0,
            ## masked weights would be set to 0 after softmax operation
            ## Thus, if the sample is shorter than other samples in mini-batch,
            ## the weight for empty time-step would be set to 0
            
            weight.masked_fill_(mask, -float("inf"))
            
        weight = self.softmax(weight)
        
        ## |context_vector| = (batch_size, 1, hidden_size)
        context_vector = torch.bmm(weight.unsqueeze(1), h_src)
        
        return context_vector

#### Attention을 위한 마스크 생성

Attention class를 살펴보면 마스크의 존재 여부에 따라 `masked_fill_`함수를 수행하는 것을 볼 수 있습니다. 미니배치 내부의 문장들은 길이 서로 다를 수 있으므로, 마스킹을 통해 선택적으로 어텐션을 수행하려는 것입니다. 미니배치의 크기는 미니배치 내부의 가장 긴 문장의 길이에 좌우됩니다. 길이가 짧은 문장들은 문장의 종류 후에 Padding으로 채워집니다.

<br></br>
![](./images/10-4-4-batch1.jpg)
<br></br>

따라서 해당 미니배치가 인코더를 통과하고 디코더에서 어텐션 연산을 수행했을때 문제가 발생합니다. 어텐션 연산은 벡터 내적을 사용하여 어텐션 가중치를 계산하므로, 패딩이 존재하는 time-step에도 어텐션 가중치가 갈 수 있다는 것입니다. 즉, 단어가 존재하는 않는 곳인데 디코더에게 쓸데없는 정보를 넘겨주게 됩니다. 따라서 해당 time-step에는 어텐션 가중치를 추가적으로 다시 0으로 만들어주는 작업이 필요합니다.

<br></br>
![](./images/10-4-4-batch2.jpg)
<br></br>

앞의 어텐션 코드에서는 `softmax`를 수행하기전에 `masked_fill_` 함수를 따라서 마스크에 따라 음의 무한대의 값으로 바꿔줍니다. 그럼 해당 time-step의 softmax결과는 0이 될 것이고, 우리는 패딩이 있는 위치에는 어텐션 가중치를 주지 않게 될 것입니다.

다음은 seq2seq 클래스 내부에 정의된 마스크를 생성하는 함수입니다.

<br></br>
```python
def generate_mask(self, x, length):
    mask = []
    
    max_length = max(length)
    
    for l in length:
        if max_length - l > 0:
            ## If the length is shorter than maximum length among samples,
            ## set last few values to be 1s to remove attention weight
            mask += [torch.cat([x.new_ones(1, l).zero_(),
                                x.new_ones(1, (max_length - l))], dim = -1)]
            
        else:
            ## If the length of the sample equals to maximum length among samples,
            ## set every value in mask to be 0
            mask += [x.new_ones(1, l).zero_()]
            
    mask = torch.cat(mask, dim = 0).byte()
    
    return mask   
```
<br></br>

#### Encoder Class

In [3]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    
    def __init__(self, word_vec_dim, hidden_size, n_layers = 4, dropout_p = 0.2):
        super(Encoder, self).__init__()
        
        ## hidden_size is half of original hidden_size, because it is bidirectional
        self.rnn = nn.LSTM(word_vec_dim,
                           int(hidden_size / 2),
                           num_layers = n_layers,
                           dropout = dropout_p,
                           bidirectional = True,
                           batch_first = True)
        
        def forward(self, emb):
            # |emb| = (batch_size, length, word_vec_dim)
            
            if isinstance(emb, tuple):
                x, lengths = emb
                x = pack(x, lengths.tolist(), batch_first = True)
                
            else:
                x = emb
                
            ## |y| = (batch_size, length, hidden_size)
            ## |h[0]| = (num_layers * 2, batch_size, hidden_size / 2)
            y, h = self.rnn(x)
            
            if isinstance(emb, tuple):
                y, _ = unpack(y, batch_first = True)
                
            return y, h

#### Decoder Class

Decoder는 한 time-step씩 입력받아 동작하도록 설계되어 있습니다. 또한 인코더와 달리 이전 time-step의 $\tilde{h}$를 받아 이어붙여 RNN의 입력을 만들어줍니다. 따라서 디코더 RNN의 입력 크기는 단순히 단어 임베딩 벡터의 크기가 아니라 hidden_size가 더해진 값이 선언됩니다.

In [4]:
class Decoder(nn.Module):
    
    def __init__(self, word_vec_dim, hidden_size, n_layers = 4, dropout_p = 0.2):
        super(Decoder, self).__init__()
        
        self.rnn = nn.LSTM(word_vec_dim + hidden_size,
                           hidden_size,
                           num_layers = n_layers,
                           dropout = dropout_p,
                           bidirectional = False,
                           batch_first = True)
        
    
    def forward(self, emb_t, h_t_1_tilde, h_t_1):
        ## |emb_t| = (batch_size, 1, word_vec_dim)
        ## |h_t_1_tilde| = (batch_size, 1, hidden_size)
        ## |h_t_1[0]| = (n_layers, batch_size, hidden_size)
        batch_size = emb_t.size(0)
        hidden_size = h_t_1[0].size(-1)
        
        if h_t_1_tilde is None:
            ## If this is the first time-step
            h_t_1_tilde = emb_t.new(batch_size, 1, hidden_size).zero_()
            
        ## Input feeding
        x = torch.cat([emb_t, h_t_1_tilde], dim = -1)
        
        ## Unlike encoder, decoder must take an input  sequentially.
        y, h = self.rnn(x, h_t_1)
        
        return y, h

#### Generator Class

In [5]:
class Generator(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Generator, self).__init__()
        
        self.output = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = -1)
        
    def forward(self, x):
        ## |x| = (batch_size, length, hidden_size)
        
        ## |y| = (batch_size, length, output_size)
        y = self.softmax(self.output(X))
        
        return y

#### Seq2seq Class

In [6]:
class Seq2Seq(nn.Module):
    
    def __init__(self,
                 input_size,
                 word_vec_dim,
                 hidden_size,
                 output_size,
                 n_layers = 4,
                 dropout_p = 0.2):
        
        self.input_size = input_size
        self.word_vec_dim = word_vec_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        super(Seq2Seq, self).__init__()
        
        self.emb_src = nn.Embedding(input_size, word_vec_dim)
        self.emb_dec = nn.Embedding(output_size, word_vec_dim)
        
        self.encoder = Encoder(word_vec_dim,
                              hidden_size,
                              n_layers = n_layers,
                              dropout_p = dropout_p)
        
        self.decoder = Decoder(word_vec_dim,
                               hidden_size,
                               n_layers = n_layers,
                               dropout_p = dropout_p)
        
        self.attn = Attention(hidden_size)
        
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.tanh = nn.Tanh()
        self.generator = Generator(hidden_size, output_size)        

#### 인코더의 은닉 상태를 디코더의 은닉 상태로 변환하기

인코더는 양방향 LSTM으로 구성됩니다. 양방향 LSTM은 방향의 개수만큼 은닉 상태를 갖기때문에, 단방향 LSTM과 다른 은닉 상태 형태를 가집니다. 즉, 양방향 LSTM은 **"2 x 계층의 개수"** 만큼의 은닉 상태를 가집니다. 인코더의 마지막 time-step의 은닉 상태는 디코더의 초기 은닉 상태가 될 것이므로, 그 크기를 맞춘 다음에 디코더의 은닉 상태로 넣어주어야 합니다. 이를 위해 우리는 애초에 인코더의 hidden_size를 원래 반으로 사용했습니다. 다음처럼 텐서의 위치를 조작하여 디코더의 은닉 상태에 알맞은 형태로 만들어줄 수 있습니다.

<br></br>
![](./images/10-5-1-hiddenstate.jpg)
<br></br>